# シンプルなデータ取得テスト

データベース接続とデータ取得の基本テストを行います。

In [ ]:
# 基本ライブラリのインポート
import pandas as pd
import sys
import os

print("Step 1: Basic libraries imported")

In [ ]:
# パス設定
current_dir = os.getcwd()
config_path = os.path.join(os.path.dirname(current_dir), 'config')
sys.path.insert(0, config_path)

print(f"Step 2: Config path set to: {config_path}")
print(f"Current directory: {current_dir}")

In [ ]:
# 設定ファイルのインポートテスト
try:
    from data_utils import DataFetcher
    print("Step 3: DataFetcher import - SUCCESS")
except ImportError as e:
    print(f"Step 3: DataFetcher import - FAILED: {e}")
    raise

In [ ]:
# データベース接続テスト
try:
    fetcher = DataFetcher()
    print("Step 4: DataFetcher initialization - SUCCESS")
except Exception as e:
    print(f"Step 4: DataFetcher initialization - FAILED: {e}")
    raise

In [ ]:
# 接続テスト
try:
    connection_result = fetcher.test_connection()
    if connection_result:
        print("Step 5: Database connection test - SUCCESS")
    else:
        print("Step 5: Database connection test - FAILED")
except Exception as e:
    print(f"Step 5: Database connection test - ERROR: {e}")

In [ ]:
# 少量のデータ取得テスト（7日分）
try:
    print("Step 6: Fetching small amount of data (7 days)...")
    copper_data = fetcher.get_copper_prices(days=7)
    
    print(f"Data retrieved: {len(copper_data)} records")
    
    if not copper_data.empty:
        print("\nData preview:")
        print(copper_data.head())
        
        print("\nData info:")
        print(f"Columns: {list(copper_data.columns)}")
        print(f"Date range: {copper_data['TradeDate'].min()} to {copper_data['TradeDate'].max()}")
        print(f"Exchanges: {copper_data['ExchangeCode'].unique()}")
        print(f"Tenor types: {copper_data['TenorTypeName'].unique()}")
        
        print("\nStep 6: Data fetch - SUCCESS")
    else:
        print("Step 6: Data fetch - WARNING: No data returned")
        
except Exception as e:
    print(f"Step 6: Data fetch - ERROR: {e}")

In [ ]:
# データ詳細確認
if 'copper_data' in locals() and not copper_data.empty:
    print("Step 7: Data analysis")
    print("\nData types:")
    print(copper_data.dtypes)
    
    print("\nNull values:")
    print(copper_data.isnull().sum())
    
    print("\nUnique values per column:")
    for col in copper_data.columns:
        unique_count = copper_data[col].nunique()
        print(f"  {col}: {unique_count} unique values")
        
    print("\nStep 7: Data analysis - COMPLETE")
else:
    print("Step 7: No data available for analysis")

## アウトライト価格の可視化

取得したデータからアウトライト（現物・先物）価格をチャートで表示します。

In [ ]:
# 可視化ライブラリのインポート
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

print("Step 8: Visualization libraries imported")

# 日本語フォント設定（エラー回避）
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# より多くのヒストリカルデータを取得（30日分）
try:
    print("Step 9: Fetching historical data (30 days)...")
    historical_data = fetcher.get_copper_prices(days=30)
    
    print(f"Historical data retrieved: {len(historical_data)} records")
    print(f"Date range: {historical_data['TradeDate'].min()} to {historical_data['TradeDate'].max()}")
    
    # アウトライト価格のフィルタリング（Cash, 3M Futures, Generic 1st Future）
    outright_tenors = ['Cash', '3M Futures', 'Generic 1st Future']
    outright_data = historical_data[historical_data['TenorTypeName'].isin(outright_tenors)].copy()
    
    print(f"Outright data filtered: {len(outright_data)} records")
    print(f"Available tenors: {outright_data['TenorTypeName'].unique()}")
    
    print("Step 9: Historical data fetch - SUCCESS")
    
except Exception as e:
    print(f"Step 9: Historical data fetch - ERROR: {e}")
    historical_data = pd.DataFrame()
    outright_data = pd.DataFrame()

In [ ]:
# アウトライト価格の時系列チャート作成
if not outright_data.empty:
    print("Step 10: Creating outright price chart...")
    
    # 日付をdatetimeに変換
    outright_data['TradeDate'] = pd.to_datetime(outright_data['TradeDate'])
    
    # 価格チャートの作成
    plt.figure(figsize=(14, 8))
    
    # 限月タイプごとに線を描画
    colors = {'Cash': '#1f77b4', '3M Futures': '#ff7f0e', 'Generic 1st Future': '#2ca02c'}
    
    for tenor in outright_data['TenorTypeName'].unique():
        tenor_data = outright_data[outright_data['TenorTypeName'] == tenor]
        if not tenor_data.empty:
            plt.plot(tenor_data['TradeDate'], tenor_data['LastPrice'], 
                    label=tenor, linewidth=2, color=colors.get(tenor, '#333333'), marker='o', markersize=4)
    
    # チャートの設定
    plt.title('Copper Outright Price Historical Chart (30 Days)', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Price (USD/MT)', fontsize=12)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    
    # 日付軸の書式設定
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=3))
    plt.xticks(rotation=45)
    
    # レイアウト調整
    plt.tight_layout()
    plt.show()
    
    print("Step 10: Outright price chart - SUCCESS")
    
else:
    print("Step 10: No outright data available for charting")

In [ ]:
# 価格統計の表示
if 'outright_data' in locals() and not outright_data.empty:
    print("Step 11: Price statistics analysis...")
    print("\n" + "="*50)
    print("OUTRIGHT PRICE STATISTICS")
    print("="*50)
    
    for tenor in outright_data['TenorTypeName'].unique():
        tenor_data = outright_data[outright_data['TenorTypeName'] == tenor]
        if not tenor_data.empty:
            latest_price = tenor_data['LastPrice'].iloc[-1]
            min_price = tenor_data['LastPrice'].min()
            max_price = tenor_data['LastPrice'].max()
            avg_price = tenor_data['LastPrice'].mean()
            
            print(f"\n{tenor}:")
            print(f"  Latest Price: {latest_price:,.2f} USD/MT")
            print(f"  30-Day Range: {min_price:,.2f} - {max_price:,.2f} USD/MT")
            print(f"  30-Day Average: {avg_price:,.2f} USD/MT")
            
            # 30日間の変動計算
            if len(tenor_data) > 1:
                first_price = tenor_data['LastPrice'].iloc[0]
                price_change = latest_price - first_price
                price_change_pct = (price_change / first_price) * 100
                print(f"  30-Day Change: {price_change:+,.2f} USD/MT ({price_change_pct:+.2f}%)")
    
    print("\nStep 11: Price statistics analysis - COMPLETE")
else:
    print("Step 11: No data available for statistics")

In [ ]:
# 最終サマリー（修正版）
print("="*60)
print("ENHANCED DATA TEST WITH VISUALIZATION SUMMARY")
print("="*60)

if 'copper_data' in locals() and not copper_data.empty:
    print(f"✓ Database connection: SUCCESS")
    print(f"✓ Basic data retrieval: SUCCESS ({len(copper_data)} records)")

if 'historical_data' in locals() and not historical_data.empty:
    print(f"✓ Historical data retrieval: SUCCESS ({len(historical_data)} records)")
    
if 'outright_data' in locals() and not outright_data.empty:
    print(f"✓ Outright price filtering: SUCCESS ({len(outright_data)} records)")
    print(f"✓ Price visualization: SUCCESS")
    print(f"✓ Statistical analysis: SUCCESS")
    
    print(f"\nData Summary:")
    print(f"  - Date Range: {historical_data['TradeDate'].min()} to {historical_data['TradeDate'].max()}")
    print(f"  - Available Tenors: {', '.join(outright_data['TenorTypeName'].unique())}")
    print(f"  - Total Data Points: {len(outright_data)}")
    
    print("\n🎉 Ready for advanced analysis and visualization!")
else:
    print("✗ Some visualization steps failed - check the steps above")

print("="*60)